问题描述
绿色物流中心选址问题：某电商公司需在5个候选位置中选择建立物流中心，需考虑：

建设成本（不同位置成本不同）
覆盖人口（不同位置覆盖不同区域人口）
碳排放量（不同位置运输距离不同）
目标：在预算限制内，最大化覆盖人口，同时满足碳排放限制

决策变量：yᵢ = {0,1} 表示是否在位置i建物流中心

In [ ]:
import pulp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Circle

In [ ]:
# 1. 定义问题数据
# 候选位置信息
locations = ['A', 'B', 'C', 'D', 'E']
costs = [50, 70, 40, 60, 55]     # 建设成本(万元)
populations = [80, 100, 60, 90, 75]  # 覆盖人口(千人)
emissions = [12, 15, 8, 14, 10]    # 碳排放(吨/年)

# 问题参数
budget = 150  # 总预算(万元)
max_emission = 30  # 最大允许碳排放(吨/年)
min_facilities = 1  # 最少建设数量
max_facilities = 3  # 最多建设数量

In [ ]:
# 2. 创建问题实例（最大化问题）
prob = pulp.LpProblem("Green_Facility_Location", pulp.LpMaximize)

In [ ]:
# 3. 定义决策变量（0-1变量）
y = pulp.LpVariable.dicts("Location", 
                         locations, 
                         cat=pulp.LpBinary)

In [ ]:
# 4. 定义目标函数：最大化覆盖人口
prob += pulp.lpSum([populations[i] * y[locations[i]] 
                   for i in range(len(locations))]), "Total_Population"

In [ ]:
# 5. 添加约束条件
# 5.1 预算约束
prob += pulp.lpSum([costs[i] * y[locations[i]] 
                   for i in range(len(locations))]) <= budget, "Budget_Constraint"

# 5.2 碳排放约束
prob += pulp.lpSum([emissions[i] * y[locations[i]] 
                   for i in range(len(locations))]) <= max_emission, "Emission_Constraint"

# 5.3 最少/最多建设数量约束
prob += pulp.lpSum(y) >= min_facilities, "Min_Facilities"
prob += pulp.lpSum(y) <= max_facilities, "Max_Facilities"

In [ ]:
# 6. 打印问题定义（调试用）
print("=== 问题定义 ===")
print(prob)

In [ ]:
# 7. 求解问题
solver = pulp.PULP_CBC_CMD(msg=True)  # 可以使用其他求解器如: CPLEX, Gurobi
prob.solve(solver)

In [ ]:
# 8. 分析和展示结果
print("\n" + "="*50)
print("求解结果:")
print("="*50)
print(f"状态: {pulp.LpStatus[prob.status]}")
print(f"最大覆盖人口: {pulp.value(prob.objective):.0f} 千人")

# 提取选定的位置
selected = [loc for loc in locations if y[loc].varValue == 1]
print(f"选定的物流中心位置: {selected}")
print("\n详细结果:")

# 创建结果表格
results = []
total_cost = 0
total_emission = 0
for i, loc in enumerate(locations):
    if y[loc].varValue == 1:
        cost = costs[i]
        pop = populations[i]
        emission = emissions[i]
        total_cost += cost
        total_emission += emission
        results.append([loc, "是", cost, pop, emission])
    else:
        results.append([loc, "否", costs[i], populations[i], emissions[i]])

# 显示结果表格
df = pd.DataFrame(results, 
                 columns=["位置", "选定", "建设成本(万元)", "覆盖人口(千人)", "碳排放(吨/年)"])
print(df.to_string(index=False))

# 显示结果表格
df = pd.DataFrame(results, 
                 columns=["位置", "选定", "建设成本(万元)", "覆盖人口(千人)", "碳排放(吨/年)"])
print(df.to_string(index=False))

print("\n总体指标:")
print(f"- 总建设成本: {total_cost} 万元 (预算: {budget} 万元)")
print(f"- 总碳排放: {total_emission} 吨/年 (上限: {max_emission} 吨/年)")
print(f"- 总覆盖人口: {pulp.value(prob.objective):.0f} 千人")

In [ ]:
# 9. 可视化选址结果
plt.figure(figsize=(12, 8))

# 模拟位置坐标（实际应用中应使用真实地理坐标）
np.random.seed(42)
coords = {
    'A': (2, 8),
    'B': (5, 9),
    'C': (8, 7),
    'D': (3, 4),
    'E': (7, 3)
}

# 绘制城市区域（模拟）
city_x = [1, 1, 10, 10, 1]
city_y = [1, 10, 10, 1, 1]
plt.plot(city_x, city_y, 'k-', alpha=0.3, label='城市边界')

# 绘制人口密度（模拟）
x = np.linspace(0, 11, 20)
y = np.linspace(0, 11, 20)
X, Y = np.meshgrid(x, y)
Z = np.exp(-0.1*((X-5)**2 + (Y-5)**2)) + 0.5*np.exp(-0.2*((X-8)**2 + (Y-3)**2))
plt.contourf(X, Y, Z, 15, cmap='Reds', alpha=0.3)
plt.colorbar(label='相对人口密度')

# 绘制候选位置
for loc, (x, y) in coords.items():
    if loc in selected:
        plt.scatter(x, y, s=300, c='green', marker='s', edgecolor='black', linewidth=2, 
                   label=f'选定位置 {loc}' if loc == selected[0] else "")
        # 绘制覆盖范围
        circle = Circle((x, y), 2.5, color='green', alpha=0.1)
        plt.gca().add_patch(circle)
    else:
        plt.scatter(x, y, s=200, c='gray', marker='o', edgecolor='black', 
                   label='未选位置' if loc == 'A' else "")
        
# 添加标签
for loc, (x, y) in coords.items():
    plt.text(x+0.3, y+0.3, loc, fontsize=12, fontweight='bold')

plt.title('物流中心选址结果', fontsize=15)
plt.xlabel('X坐标 (km)', fontsize=12)
plt.ylabel('Y坐标 (km)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(loc='best')
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# 10. 敏感性分析：预算变化的影响
budget_range = range(80, 201, 20)
max_populations = []
num_facilities = []

for b in budget_range:
    # 创建新问题
    prob_sens = pulp.LpProblem("Sensitivity_Analysis", pulp.LpMaximize)
    
    # 决策变量
    y_sens = pulp.LpVariable.dicts("Location", locations, cat=pulp.LpBinary)
    
    # 目标函数
    prob_sens += pulp.lpSum([populations[i] * y_sens[locations[i]] for i in range(len(locations))])
    
    # 约束
    prob_sens += pulp.lpSum([costs[i] * y_sens[locations[i]] for i in range(len(locations))]) <= b
    prob_sens += pulp.lpSum([emissions[i] * y_sens[locations[i]] for i in range(len(locations))]) <= max_emission
    prob_sens += pulp.lpSum(y_sens) >= min_facilities
    prob_sens += pulp.lpSum(y_sens) <= max_facilities
    
    # 求解
    prob_sens.solve(solver)
    
    # 记录结果
    max_populations.append(pulp.value(prob_sens.objective))
    num_facilities.append(sum(y_sens[loc].varValue for loc in locations))

# 绘制敏感性分析图
plt.figure(figsize=(10, 6))
plt.plot(budget_range, max_populations, 'bo-', linewidth=2, label='覆盖人口')
plt.bar(budget_range, num_facilities, width=10, alpha=0.3, color='green', label='建设数量')

plt.title('预算敏感性分析', fontsize=15)
plt.xlabel('预算 (万元)', fontsize=12)
plt.ylabel('覆盖人口 (千人)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()

# 添加数据标签
for i, (b, pop) in enumerate(zip(budget_range, max_populations)):
    plt.annotate(f'{int(pop)}', (b, pop+2), textcoords="offset points", 
                 xytext=(0,5), ha='center')

plt.tight_layout()
plt.show()

print("\n" + "="*50)
print("预算敏感性分析结果:")
print("="*50)
for b, pop, num in zip(budget_range, max_populations, num_facilities):
    print(f"预算 = {b:3d}万元 | 最大覆盖人口 = {pop:3.0f}千人 | 建设数量 = {int(num)}")